# Lab - Collaborative Recommender System

In this lab you will implement the collaborative ﬁltering learning algorithm and apply it to a dataset of movie ratings. 

MovieLens 100k Dataset from GroupLens Research (https://grouplens.org/datasets/movielens/). 

This dataset consists of ratings on a scale of 1 to 5. The dataset has $n_u$ = 943 users, and
$n_m$ = 1682 movies. 

In [ ]:
#Load relevant libraries

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd

  ## Load the data
Load the matrices $Y$ and $R$ from matlab file *ex8_movies.mat*. 
 
$Y$ (*num_movies × num_users*)  stores the ratings $y^{(i,j)}$
(from 1 to 5). 

$R$ is an binary-valued indicator matrix, where $R(i, j)$ = 1 if user $j$ gave a rating to movie $i$, and $R(i, j)$ = 0 otherwise. 

The objective of collaborative ﬁltering is to predict movie ratings for the movies
that users have not yet rated, that is, the entries with $R(i, j) = 0$. This will allow us to recommend the movies with the highest predicted ratings to the user.

In [ ]:
# Load Y and R

Y = ?    # 1682 X 943 matrix, containing ratings (1-5) of 1682 movies on 943 user

R =  ?   # 1682 X 943 matrix, R(i,j) = 1 if and only if user j give rating to movie i

# Confirm the shapes of Y and R

?

In [ ]:
# How many users gave a score for the first movie ? ANSWER = 452
? 

# how many users gave a score for the second movie ? ANSWER= 131
?

 # How many users gave a score for the last movie ? ANSWER = 1 
?


In [ ]:
#Compute the average movie rating for the ﬁrst movie (Toy Story). ANSWER: 3.88

?

### Collaborative Filtering Learning Algorithm

The collaborative ﬁltering algorithm for movie recommendations considers that the movies have features, for example x1=romance, x2=action, x3=comedy, etc. In general the movies have $n$ different features, represented as a set of n-dimensional feature vectors X= $[ x^{(1)},... x^{(n_m)}] $. 

The model that will predict what is the rating $y(i,j)$ for movie $i$ given by user $j$   is formulated as a Linear Regression between the movie features and the user parameters Theta= $[ θ^{(1)}, ...,θ^{(n_u)}] $, that is 

$ y(i,j)=(θ^{(j)})^{T}x^{(i)}$ =>  **Linear Regression model for computing the scores for movie $i$ given by user $j$**

Given a dataset of ratings produced
by **some users on some movies**, the model will try to learn simultaneously both vectors  

 X=$ [x^{(1)},... x^{(n_m)}]$ and Theta=$[θ^{(1)}, ...,θ^{(n_u)}]$
 
 that produce the best ﬁt (minimize the
error between the predictions $y(i,j)$ and the real scores given by the users).


**The collaborative filtering cost function with regularization terms is given by**

$J(x^{(1)},...,x^{(n_m)},\theta^{(1)},...,\theta^{(n_u)}) = \frac{1}{2} \sum_{((i,j): r(i,j)=1)}((\theta^{(j)})^Tx^{(i)} - y^{(i,j)})^2 + (\frac{\lambda}{2} \sum^{n_u}_{j=1}\sum^n_{k=1} (\theta^{(j)}_k)^2) + (\frac{\lambda}{2} \sum^{n_m}_{i=1}\sum^n_{k=1} (x^{(i)}_k)^2)$

**The collaborative filtering gradients of the cost function $J$ (with regularization) are given by**

$\frac{\partial J}{\partial x^{(i)}_k} = \sum ((\theta^{(j)})^Tx^{(i)} - y^{(i,j)})\theta_k^{(j)} +\lambda x^{(i)}_k$

$\frac{\partial J}{\partial \theta^{(j)}_k} = \sum ((\theta^{(j)})^Tx^{(i)} - y^{(i,j)})x_k^{(i)} +\lambda \theta^{(j)}_k$

In [ ]:
def  cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda):
    """
    Returns the cost and gradient for the collaborative filtering problem
    
    Lambda - regularization parameter
    
    """
        
    # Unfold the params
    X = params[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = params[num_movies*num_features:].reshape(num_users,num_features)
    
    predictions =  X @ Theta.T
    err = (predictions - Y)
    J = 1/2 * np.sum((err**2) * R)
    
    #compute regularized cost function
    reg_X =  Lambda/2 * np.sum(Theta**2)
    reg_Theta = Lambda/2 *np.sum(X**2)
    reg_J = J + reg_X + reg_Theta
    
    # Compute gradient
    X_grad = err*R @ Theta
    Theta_grad = (err*R).T @ X
    grad = np.append(X_grad.flatten(),Theta_grad.flatten())
    
    # Compute regularized gradient
    reg_X_grad = X_grad + Lambda*X
    reg_Theta_grad = Theta_grad + Lambda*Theta
    reg_grad = np.append(reg_X_grad.flatten(),reg_Theta_grad.flatten())
    
    return J, grad, reg_J, reg_grad

In function *cofiCostFunc*: 
 
$X$ (num_movies × n) is the matrix of movie features (also called movie parameters), $n$ is the number of features,  $i$ row of $X$ corresponds to the feature vector $x^{(i)}$ of the $i$ movie.

$Theta$ (num_users × n) is the matrix of user parameters, $j$ row of $Theta$ corresponds to the parameter vector $θ^{(j)}$, of the $j$ user. 

Both $x^{(i)}$ and $θ^{(j)}$ are n-dimensional vectors, $x^{(i)} ∈ R^{n}$ and $θ^{(j)}∈R^{n}$ .

$X$ and $Theta$ will be inicialized randomly latter on, however, in order to test the *cofiCostFunc* function, we have saved some parameters $X$ and $Theta$ in the matlab file *ex8_movieParams.mat*. 


In [ ]:
#Load X and Theta from ex8_movieParams.mat and print their dimensions. 

X = ? #  (num_movies X num_features)

Theta = ? #  (num_users X num_features) 

#How many features have the movies ?
?

In [ ]:
# In order to speed up the test, data set size is reduced 
num_users, num_movies, num_features = 4,5,3

#Extract from X, Theta, Y, and R only data coresponding to 
#the new number of users, number of movies, number of features 

X_test = ? #(5,3)
Theta_test= ?  #(4,3)
Y_test = ? #(5,4)
R_test = ? # (5,4)

# Due to the simultaneous optimization of X and Theta, 
#all elements of X and Theta are flattened and appended into a single vector
params = np.append(X_test.flatten(),Theta_test.flatten())

# Compute the cost function (without regularization).  ANSWER: J=22.22
print("Cost at loaded parameters (lambda = 0):")
?

# Compute the cost function (with regularization). ANSWER: J=31.34
print("Cost at loaded parameters (lambda = 1.5):")
?

### Data set of MOVIES 

The list of all movies and their index in the dataset are listed in the ﬁle *movie_idx.txt*

In [ ]:
# load the movie list
movieList = open("movie_ids.txt","r").read().split("\n")[:-1]

#for Linux users
#movieList = open("movie_ids.txt","r", encoding='ISO-8859-1').read().split("\n")[:-1]

#How many movies are collected ?
num_movies= ?

# see the movie list
movieList

### Give your own preferences  for some of the movies 
We have ﬁlled out some values according to our own preferences, but you can enter your own movie preferences, so that later when the algorithm runs, you can get your own movie recommendations!

In [ ]:
# Initialize my ratings
my_ratings = np.zeros((num_movies,1))

# Create own ratings
my_ratings[0] = 4 
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[82]= 4
my_ratings[225] = 5
my_ratings[354]= 5

print("New user ratings:\n")
for i in range(len(my_ratings)):
    if my_ratings[i]>0:
        print("Rated",int(my_ratings[i]),"for index",movieList[i])

Add the additional ratings in the original dataset Y and R

In [ ]:
Y = np.hstack((my_ratings,Y))
R =np.hstack((my_ratings!=0,R))

#Confirm that the number of users increased by one
?

### Learning movie recommendations

Normalize the ratings Y. 

Generate random initial values for X and Theta. 

Call Gradient Descent to optimize X and Theta. 

In [ ]:
def normalizeRatings(Y, R):
    """
    normalized Y so that each movie has a rating of 0 on average, and returns the 
    mean rating in Ymean.
    """
  
    m = ?  # number of movies
    n= ? # number of users
    
    #Inicialize Ymean as column vector of 0 with m elements 
    #Inicialize Ynorm as matrix with the same dimenssion as Y 
    Ymean = ?
    Ynorm = ?
    
    for i in range(m):
        Ymean[i] = np.sum(Y[i,:])/np.count_nonzero(R[i,:])
        Ynorm[i,R[i,:]==1] = Y[i,R[i,:]==1] - Ymean[i]
        
    return Ynorm, Ymean

In [ ]:
def gradientDescent(initial_parameters,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda):
    """
    alpha - learning rate
    Optimize X and Theta
    """
    # unfold the initial parameters (consult function cofiCostFunc)
    X = ?
    Theta =  ?
    
    J_history =[]
    
    for i in range(num_iters):
        #Append into a single vector params X and Theta (see the code above)
        params = ?
        cost, grad = cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda)[2:]
        
        # unfold grad
        X_grad = grad[:num_movies*num_features].reshape(num_movies,num_features)
        Theta_grad = grad[num_movies*num_features:].reshape(num_users,num_features)
        
    #Update the trainable parameters X and Theta with the classical gradient descent method
        X = ?
        
        Theta = ?
        
        J_history.append(cost)
    
    #Append into a single vector paramsFinal the updated X and Theta
    paramsFinal = ?
    return paramsFinal , J_history

In [ ]:
# Normalize Ratings
Ynorm, Ymean = ?

In [ ]:
# number of users
num_users = ?

# number of movies
num_movies = ?

num_features = 10

# Generate randomly initial values for the matrices X and Theta 
#(use for example np.random.randn())
X = ?
Theta = ?

#Append into a single vector params X and Theta 
initial_parameters = ?

Lambda = 10
num_iters=400
alpha=0.001

# Optimize parameters using Gradient Descent, use the normalized Ynorm
paramsFinal, J_history = ?

### Plot the cost Function

In [ ]:
?


In [ ]:
#unfold paramsFinal (consult function cofiCostFunc)
X = ?
Theta = ?

# Predict all ratings of num_users for num_movies
p = X @ Theta.T

# Extract from p only the recomendations for the added user (the first one)
# Reshape because of rank one problem 

my_scores= ?

my_predictions = my_scores + Ymean


In [ ]:
df = pd.DataFrame(np.column_stack((my_predictions,np.array(movieList))))

df.sort_values(by=[0],ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)

#Extract only the top 10 recommented movies for the added user 
?